In [1]:
import pandas as pd
from os import path

path_partB = "C:\\Users\\bm-\\Downloads\\Parte B\\Parte B"

In [2]:
df_b = pd.read_csv(path.join(path_partB, "Examen Parte B.csv"), sep=",", header=0)  #########

In [3]:
print(df_b.count(), "\n")
print(df_b.dtypes, "\n")
df_b.head()

Comercio Id                  2000
Mes Operacion                2000
Estado                       2000
Moneda                       2000
# Transacciones              2000
Monto Transaccion            2000
Monto Contracargo (Venta)    2000
dtype: int64 

Comercio Id                   int64
Mes Operacion                object
Estado                       object
Moneda                       object
# Transacciones              object
Monto Transaccion            object
Monto Contracargo (Venta)    object
dtype: object 



,Comercio Id,Mes Operacion,Estado,Moneda,# Transacciones,Monto Transaccion,Monto Contracargo (Venta)
0,0,01/07/21,IN_PROGRESS,MXN,"3,679,780","11,928,910,467",0
1,0,01/07/21,COMPLETED,MXN,"12,366","23,472,915",0
2,0,01/07/21,FAILED,MXN,"5,115","7,124,189.72",0
3,0,01/08/21,IN_PROGRESS,MXN,"2,536,306","8,193,985,268",0
4,0,01/08/21,CANCELLED,MXN,"876,854","2,786,923,528",0


In [5]:
df_b1 = df_b.copy() #########

In [6]:
df_b1[["# Transacciones", "Monto Transaccion", "Monto Contracargo (Venta)"]] = \
df_b1[["# Transacciones", "Monto Transaccion", "Monto Contracargo (Venta)"]].replace(to_replace="[^\d\.]", value="", regex=True)

In [7]:
df_b1.head()

,Comercio Id,Mes Operacion,Estado,Moneda,# Transacciones,Monto Transaccion,Monto Contracargo (Venta)
0,0,01/07/21,IN_PROGRESS,MXN,3679780,11928910467,0
1,0,01/07/21,COMPLETED,MXN,12366,23472915,0
2,0,01/07/21,FAILED,MXN,5115,7124189.72,0
3,0,01/08/21,IN_PROGRESS,MXN,2536306,8193985268,0
4,0,01/08/21,CANCELLED,MXN,876854,2786923528,0


**a)	Coloque las características en el formato apropiado.**

In [8]:
#---a
df_b1["Comercio Id"] = df_b1["Comercio Id"].astype(int)
df_b1["Mes Operacion"] = pd.to_datetime(df_b1["Mes Operacion"], format="%d/%m/%y")
df_b1["# Transacciones"] = df_b1["# Transacciones"].astype(int)
df_b1["Monto Transaccion"] = df_b1["Monto Transaccion"].astype(float)
df_b1["Monto Contracargo (Venta)"] = df_b1["Monto Contracargo (Venta)"].astype(float)

In [9]:
df_b1.dtypes

Comercio Id                           int32
Mes Operacion                datetime64[ns]
Estado                               object
Moneda                               object
# Transacciones                       int32
Monto Transaccion                   float64
Monto Contracargo (Venta)           float64
dtype: object

**b)	Filtre solo las monedas correspondientes a MXN y USD**

In [10]:
#---b
df_b2 = df_b1[(df_b1["Moneda"]=="MXN") | (df_b1["Moneda"]=="USD")].reset_index(drop=True)  #########

In [11]:
print(df_b1.groupby("Moneda").count()["Comercio Id"],"\n")
print(df_b2.groupby("Moneda").count()["Comercio Id"])

Moneda
AED       8
ARS       2
CLP      10
COP      52
EUR      18
JPY      12
MXN    1855
PKR       8
TWD       6
USD      29
Name: Comercio Id, dtype: int64 

Moneda
MXN    1855
USD      29
Name: Comercio Id, dtype: int64


**c)	Realice una transformación de los dólares a pesos de la misma forma que en el ejercicio anterior.**

In [12]:
#---c
from urllib.request import urlopen
import json

with urlopen("https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43718/datos?"+
"token=1728d77b21b641c1704df48ee020b179966b4100f187bc9c2bc83d0f671ad83b") as response:
    resp = json.loads(response.read().decode("utf8"))

data = resp["bmx"]["series"][0]["datos"]
df_tiposCambio = pd.DataFrame(data)
df_tiposCambio["fecha"] = pd.to_datetime(df_tiposCambio["fecha"], format="%d/%m/%Y")
df_tiposCambio["dato"] = pd.to_numeric(df_tiposCambio["dato"], downcast="float")

In [13]:
means_month = df_tiposCambio.groupby(pd.PeriodIndex(df_tiposCambio["fecha"], freq="M"))["dato"].mean().reset_index()
means_month

,fecha,dato
0,1991-11,3.068377
1,1991-12,3.070921
2,1992-01,3.067723
3,1992-02,3.062905
4,1992-03,3.066768
...,...,...
364,2022-03,20.556210
365,2022-04,20.108805
366,2022-05,20.030527
367,2022-06,20.023737


In [14]:
df_b3 = df_b2.copy()    #########

In [15]:
for i, row in means_month.iterrows():
    df_b3.loc[(pd.DatetimeIndex(df_b3["Mes Operacion"]).month == means_month.loc[i, "fecha"].month) & (pd.DatetimeIndex(df_b3["Mes Operacion"]).year == means_month.loc[i, "fecha"].year) & (df_b3["Moneda"] == "USD"), ["Monto Transaccion","Monto Contracargo (Venta)"]] = \
        df_b3[(pd.DatetimeIndex(df_b3["Mes Operacion"]).month == means_month.loc[i, "fecha"].month) & (pd.DatetimeIndex(df_b3["Mes Operacion"]).year == means_month.loc[i, "fecha"].year)& (df_b3["Moneda"] == "USD")][["Monto Transaccion","Monto Contracargo (Venta)"]]\
            .apply(lambda x: x*row["dato"])

In [16]:
print(df_b3.groupby("Mes Operacion")["Comercio Id"].count())

Mes Operacion
2021-07-01    966
2021-08-01    918
Name: Comercio Id, dtype: int64


In [17]:
print(means_month[means_month["fecha"] == "2021-08"])

       fecha       dato
357  2021-08  20.076136


In [18]:
print(df_b2[(df_b2["Mes Operacion"] == "2021-08-01") & (df_b2["Moneda"] == "MXN")].tail())
print("_"*7)
print(df_b3[(df_b3["Mes Operacion"] == "2021-08-01") & (df_b2["Moneda"] == "MXN")].tail())

      Comercio Id Mes Operacion       Estado Moneda  # Transacciones  \
1874          258    2021-08-01  IN_PROGRESS    MXN              895   
1875          258    2021-08-01       FAILED    MXN               49   
1879          259    2021-08-01       FAILED    MXN             1636   
1880          259    2021-08-01    COMPLETED    MXN              826   
1881          259    2021-08-01    CANCELLED    MXN                1   

      Monto Transaccion  Monto Contracargo (Venta)  
1874          332925.20                        0.0  
1875            9302.00                        0.0  
1879         1966402.39                        0.0  
1880          991902.66                     9154.0  
1881          217369.92                        0.0  
_______
      Comercio Id Mes Operacion       Estado Moneda  # Transacciones  \
1874          258    2021-08-01  IN_PROGRESS    MXN              895   
1875          258    2021-08-01       FAILED    MXN               49   
1879          259    2021

In [19]:
print(df_b2[(df_b2["Mes Operacion"] == "2021-08-01") & (df_b2["Moneda"] == "USD")].tail())
print("_"*7)
print(df_b3[(df_b3["Mes Operacion"] == "2021-08-01") & (df_b3["Moneda"] == "USD")].tail())

      Comercio Id Mes Operacion          Estado Moneda  # Transacciones  \
1425          190    2021-08-01       COMPLETED    USD             2020   
1426          190    2021-08-01  CHARGE_PENDING    USD              502   
1427          190    2021-08-01          FAILED    USD              407   
1428          190    2021-08-01           ERROR    USD               17   
1429          190    2021-08-01     IN_PROGRESS    USD                1   

      Monto Transaccion  Monto Contracargo (Venta)  
1425          776782.78                        0.0  
1426          330535.27                        0.0  
1427          290914.30                        0.0  
1428           17388.65                        0.0  
1429              65.00                        0.0  
_______
      Comercio Id Mes Operacion          Estado Moneda  # Transacciones  \
1425          190    2021-08-01       COMPLETED    USD             2020   
1426          190    2021-08-01  CHARGE_PENDING    USD              502  

In [20]:
df_b3.loc[df_b3["Moneda"] == "USD", ["Moneda"]] = "MXN"

In [21]:
print(df_b2.groupby("Moneda").count()["Comercio Id"],"\n")
print(df_b3.groupby("Moneda").count()["Comercio Id"])

Moneda
MXN    1855
USD      29
Name: Comercio Id, dtype: int64 

Moneda
MXN    1884
Name: Comercio Id, dtype: int64


**d)	¿Cuál es el monto total de transacciones completadas?**

In [22]:
#---d
print("Monto total de transacciones completadas:\n{}\n".format(
df_b3[df_b3["Estado"] == "COMPLETED"]["Monto Transaccion"].sum()
))

Monto total de transacciones completadas:
12989560769.614422



**e)	¿Cuál es el monto total de transacciones fallidas?**

In [23]:
#---e
print("Monto total de transacciones fallidas:\n{}\n".format(
df_b3[df_b3["Estado"] == "FAILED"]["Monto Transaccion"].sum()
))

Monto total de transacciones fallidas:
9193057072.705488



**f)	¿Existe algún comercio que nunca haya completado transacciones?**

In [24]:
#---f
pvT1 = df_b3.pivot_table(index="Comercio Id", columns="Estado", values="# Transacciones", aggfunc="sum").fillna(0)
pvT1[pvT1["COMPLETED"]==0][["COMPLETED"]]

Estado,COMPLETED
Comercio Id,
100,0.0
113,0.0
134,0.0


**g)	Calcule el porcentaje de transacciones completadas en cada uno de los comercios (en el total del periodo estudiado).**

In [25]:
#---g
pvT1.div(pvT1.sum(axis=1), axis=0).mul(100)[["COMPLETED"]].rename(columns={"COMPLETED":"COMPLETED PERCENTAGE FOR ALL PERIOD"})

Estado,COMPLETED PERCENTAGE FOR ALL PERIOD
Comercio Id,
0,0.337588
1,0.438457
2,10.252216
3,42.685855
4,67.148418
...,...
256,64.242088
257,99.074593
258,48.766657


**h)	Calcule el porcentaje de transacciones completadas mensuales de cada uno de los comercios y posterior a ello calcule el promedio.**

In [26]:
#---
pvT2 = df_b3.pivot_table(index=["Comercio Id", "Mes Operacion"], columns="Estado", values="# Transacciones", aggfunc="sum").fillna(0)
pvT2.div(pvT2.sum(axis=1), axis=0).mul(100)[["COMPLETED"]].rename(columns={"COMPLETED":"COMPLETED PERCENTAGE BY MONTH"})

Estado                     COMPLETED PERCENTAGE BY MONTH
Comercio Id Mes Operacion                               
0           2021-07-01                          0.334464
            2021-08-01                          0.340955
1           2021-07-01                          0.466705
            2021-08-01                          0.414036
2           2021-07-01                          9.846414
...                                                  ...
258         2021-07-01                         47.980711
            2021-08-01                         49.464668
259         2021-07-01                         39.356670
            2021-08-01                         33.536338
260         2021-07-01                         99.593023

[501 rows x 1 columns]

In [27]:
pvT2.div(pvT2.sum(axis=1), axis=0).mul(100)[["COMPLETED"]].groupby("Comercio Id").mean()\
.rename(columns={"COMPLETED":"MEAN PERCENTAGE BY Comercio Id"})

Estado,MEAN PERCENTAGE BY Comercio Id
Comercio Id,
0,0.337710
1,0.440371
2,10.299806
3,42.679549
4,67.110779
...,...
256,64.253740
257,99.068639
258,48.722690


In [28]:
pvT2[["COMPLETED"]].groupby("Comercio Id").mean()

Estado,COMPLETED
Comercio Id,
0,12029.5
1,8325.5
2,179735.5
3,629634.5
4,828614.5
...,...
256,1157.0
257,1766.5
258,860.0


**i)	Para cada uno de los meses estudiados y cada uno de los distintos Estados construya una bandera de cuando el comercio presenta operaciones en cada tipo de estado. (Utilice 0 para representar que no cuenta con ese estado y 1 para indicar que sí cuenta con él en el periodo de tiempo observado; esto se conoce como variables dummy)**

In [29]:
df_b3.pivot_table(index=["Comercio Id", "Mes Operacion"], columns="Estado", values="Moneda", aggfunc="count").fillna(0).astype(int)

Estado                     CANCELLED  CHARGE_PENDING  COMPLETED  ERROR  \
Comercio Id Mes Operacion                                                
0           2021-07-01             0               0          1      0   
            2021-08-01             1               0          1      0   
1           2021-07-01             0               0          1      0   
            2021-08-01             0               0          1      0   
2           2021-07-01             0               0          1      1   
...                              ...             ...        ...    ...   
258         2021-07-01             0               0          1      0   
            2021-08-01             0               0          1      0   
259         2021-07-01             0               0          1      1   
            2021-08-01             1               0          1      0   
260         2021-07-01             0               0          1      0   

Estado                     EXPIRED  FAILED  IN_PROGRESS  
Comercio Id Mes Operacion                                
0           2021-07-01           0       1            1  
            2021-08-01           0       1            1  
1           2021-07-01           0       1            1  
            2021-08-01           0       1            1  
2           2021-07-01           0       1            1  
...                            ...     ...          ...  
258         2021-07-01           0       1            1  
            2021-08-01           0       1            1  
259         2021-07-01           0       1            0  
            2021-08-01           0       1            0  
260         2021-07-01           0       1            0  

[501 rows x 7 columns]

**j)	Si tiene algunas observaciones anótelas a modo de conclusiones en el notebook.**

Una observación: Si se esta utilizando VSCode en el Restricted Mode, puede que no se lleguen a ver algunos resultados, para que aparezcan los resultados, solo se debe de salir del Restricted Mode confiando en la ventana. Esto solo sucede en VSCode, en el entorno web no.